In [ ]:
# Step 1: Install required libraries (Uncomment if not already installed)
!pip install -q transformers torch torchvision torchaudio gradio gtts reportlab

# Step 2: Imports
import torch
from transformers import pipeline, Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
from gtts import gTTS
import gradio as gr
import os
import time
from reportlab.pdfgen import canvas

# Step 3: Load Whisper for Speech Recognition
whisper_model = pipeline("automatic-speech-recognition", model="openai/whisper-base")

# Step 4: Load BLIP-2 for Image Captioning
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip_model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Step 5: Rule-based Diagnosis Knowledge Base
knowledge_base = [
    {
        "name": "Allergic Rash",
        "symptoms": ["rash", "red", "itchy", "hives"],
        "advice": "You may have an allergic rash. Avoid irritants. Apply an antihistamine cream. Consult a dermatologist if it spreads."
    },
    {
        "name": "Skin Infection",
        "symptoms": ["pus", "discharge", "boil", "painful lump"],
        "advice": "You may have a skin infection. Keep the area clean. Apply antiseptic. If pain or pus increases, consult a doctor."
    },
    {
        "name": "Swelling",
        "symptoms": ["swelling", "swollen", "inflammation"],
        "advice": "Swelling can be due to allergy, trauma, or infection. Apply a cold compress and elevate the area."
    },
    {
        "name": "Joint Pain",
        "symptoms": ["joint pain", "knee pain", "elbow pain", "stiffness"],
        "advice": "This may be arthritis or overuse injury. Use ice, rest, and consult an orthopedist if persistent."
    },
    {
        "name": "Sore Throat",
        "symptoms": ["sore throat", "pain while swallowing"],
        "advice": "Gargle with warm salt water. Avoid cold drinks. If symptoms persist more than 3 days, get a throat swab test."
    },
    {
        "name": "Flu",
        "symptoms": ["fever", "body ache", "headache", "fatigue", "cough", "chills"],
        "advice": "You may have the flu. Take rest, drink fluids, and consider paracetamol. Consult a doctor if symptoms worsen."
    },
    {
        "name": "COVID-19",
        "symptoms": ["dry cough", "loss of taste", "loss of smell", "shortness of breath", "fatigue", "fever"],
        "advice": "Symptoms may indicate COVID-19. Isolate, get tested, monitor oxygen levels, and consult a healthcare provider."
    },
    {
        "name": "Common Cold",
        "symptoms": ["runny nose", "sneezing", "mild cough", "sore throat"],
        "advice": "You may have a common cold. Rest, stay hydrated, and use saline nasal spray or lozenges."
    },
    {
        "name": "Burn",
        "symptoms": ["burn", "blister", "scald"],
        "advice": "Run cool water on the burn. Don’t apply ice. Use burn ointment and cover with clean cloth. Seek help if severe."
    },
    {
        "name": "Fever (General Symptom)",
        "symptoms": ["fever", "high temperature", "hot body", "chills"],
        "advice": (
            "Fever can result from various causes like viral or bacterial infection, heat exhaustion, or autoimmune disorders. "
            "Stay hydrated, monitor the temperature, and rest. "
            "Consult a doctor if fever exceeds 102°F, lasts more than 3 days, or is accompanied by other serious symptoms."
        )
    },
]

# Step 6: Rule-Based Diagnosis with Confidence Scoring
def local_rule_based_diagnosis(symptoms, image_caption):
    full_input = (symptoms + " " + image_caption).lower()
    diagnosis_list = []

    for entry in knowledge_base:
        matches = sum(symptom in full_input for symptom in entry["symptoms"])
        if matches > 0:
            confidence = int((matches / len(entry["symptoms"])) * 100)
            diagnosis_list.append(f"{entry['name']} ({confidence}% confidence):\n{entry['advice']}")

    if (
        "fever" in full_input and
        not any(x in full_input for x in ["cough", "cold", "sore throat", "runny nose", "body ache"])
    ):
        diagnosis_list.append(
            "Fever is present without cold/flu symptoms. Consider testing for malaria, dengue, or other infections. Consult a physician."
        )

    return "\n\n".join(diagnosis_list) if diagnosis_list else \
        "No specific condition matched. Please consult a certified healthcare provider for further evaluation."

# Step 7: Generate PDF Report
def create_pdf_report(symptoms, caption, diagnosis, filepath):
    c = canvas.Canvas(filepath)
    c.setFont("Helvetica", 12)
    c.drawString(30, 800, "Medical AI Assistant - Diagnosis Report")
    c.drawString(30, 780, f"Symptoms: {symptoms[:80]}")  # limit to 80 chars
    c.drawString(30, 760, f"Image Caption: {caption[:80]}")  # limit to 80 chars
    c.drawString(30, 740, "Diagnosis:")
    text = c.beginText(30, 720)
    for line in diagnosis.split("\n"):
        text.textLine(line)
    c.drawText(text)
    c.save()

# Step 8: Main Processing Function
def process_medical_input(audio_path, text_input, image, feedback, rating):
    try:
        if not audio_path and not text_input:
            return "No symptom input provided.", "Please record or type symptoms.", None, None

        if image is None:
            return "No image uploaded.", "Please upload a relevant medical image.", None, None

        # Audio to Text
        user_query = text_input
        if audio_path:
            result = whisper_model(audio_path)
            if result["text"].strip():
                user_query = result["text"]

        # Image to Caption
        pil_image = image.convert("RGB")
        inputs = processor(images=pil_image, text="Describe any visible medical symptoms", return_tensors="pt").to(blip_model.device)
        output = blip_model.generate(**inputs, max_new_tokens=100)
        image_caption = processor.tokenizer.decode(output[0], skip_special_tokens=True)

        # Diagnosis
        diagnosis = local_rule_based_diagnosis(user_query, image_caption)
        spoken_response = "Based on your symptoms and image, here’s what I found:\n" + diagnosis

        # Save Audio & PDF Report
        timestamp = str(int(time.time()))
        audio_out = f"response_{timestamp}.mp3"
        pdf_out = f"report_{timestamp}.pdf"

        gTTS(spoken_response).save(audio_out)
        create_pdf_report(user_query, image_caption, diagnosis, pdf_out)

        # Log feedback
        with open("user_feedback_log.txt", "a") as f:
            f.write(f"\nFeedback @ {timestamp}:\nHelpfulness: {feedback}, Rating: {rating}\n")

        return user_query, diagnosis, audio_out, pdf_out

    except Exception as e:
        import traceback
        traceback.print_exc()
        return "Error", str(e), None, None

# Step 9: Gradio Interface (Now with Feedback)
iface = gr.Interface(
    fn=process_medical_input,
    inputs=[
        gr.Audio(type="filepath", label="🎙 Speak your symptoms"),
        gr.Textbox(label="Or type your symptoms here (optional)"),
        gr.Image(type="pil", label="🖼 Upload a medical image"),
        gr.Radio(["Yes", "No"], label="Was this diagnosis helpful?"),
        gr.Slider(1, 5, step=1, label="Rate this assistant (1-5 ⭐)")
    ],
    outputs=[
        gr.Textbox(label="📝 Transcribed/Typed Symptoms"),
        gr.Textbox(label="💡 Suggested Diagnosis"),
        gr.Audio(type="filepath", label="🔊 Audio Advice"),
        gr.File(label="📄 Download PDF Report")
    ],
    title="🧠 Medical AI ChatBot",
    description="Upload a medical image and either speak or type your symptoms. This AI assistant gives diagnosis, audio response, and a downloadable PDF report. Feedback welcome!"
)

iface.launch(share=True)


Device set to use cuda:0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://39865dd1ccc726878f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
